In [ ]:
!pip install --upgrade datasets fsspec transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.0/201.0 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 15.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.36.0
    Uninstalling huggingface-hub-0.36.0:
      Successfully uninstalled huggingface-hub-0.36.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uni

In [ ]:
from datasets import load_dataset
from transformers import BertTokenizer,BertForSequenceClassification,Trainer,TrainingArguments

In [ ]:
dataset=load_dataset('imdb')


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
dataset['train'][13000]

{'text': "Visually stunning and full of Eastern Philosophy, this amazing martial arts fantasy is brought to you by master director Tsui Hark, the man behind some of the best films Hong Kong cinema has produced. The special effects are beautiful and imaginative. The plot is a bit on the cerebral side, but is a refreshing change from films that treat their audience as if they were morons. If thinking is not your forte, however, this may not be your movie. Maybe you should go see the latest from the Hollywood studio's no brain club, but if you are looking for something more, he's where you will find it.",
 'label': 1}

In [ ]:
#dataset

In [ ]:
train_dataset_label_0=dataset['train'].select(range(1500))
test_dataset_label_0=dataset['test'].select(range(250))
train_dataset_label_1=dataset['train'].select(range(13000,14500))
test_dataset_label_1=dataset['test'].select(range(13000,13250))

In [ ]:
from datasets import concatenate_datasets
train_dataset = concatenate_datasets([train_dataset_label_0, train_dataset_label_1])
test_dataset = concatenate_datasets([test_dataset_label_0, test_dataset_label_1])


In [ ]:
train_dataset[2020]

{'text': 'I watched this movie alongwith my complete family of Nine. Since my younger brother has recently got married, we could connect with the goings-on. The movie stands out for the classical touch given to the romance of the engaged couple. Thankfully this time all Indian locales like Ranikhet Almora etc have been used, which have been already visited by most of the urbanites, hence adding to the connection with movie. The dialogues are much better than those in the "Umrao Jaan Ada" - a supposedly dialogue based movie. The background music is augmenting the "soft focus" of the movie. It somehow remind me of VV Chopra\'s "Kareeb", in which neha and to some extent Bobby did full justice to the character. Same here, in that the lead pair does not disappoint in any department-looks or acting. The Supporting cast are too good. I rate the actress playing the role of Bhabhi in the front league. The situations of family interactions portrayed are real and you smile when you find yourself 

In [ ]:
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def tokenizer_fun(examples):
  return tokenizer(examples['text'],padding='max_length',truncation=True,max_length=256)


In [ ]:
def preprocess(ds):
  ds=ds.map(tokenizer_fun,batched=True,remove_columns=['text'])
  ds=ds.rename_column('label','labels')
  ds.set_format(type='torch',columns=['input_ids','attention_mask','labels'])
  return ds


In [ ]:
train_dataset=preprocess(train_dataset)
test_dataset=preprocess(test_dataset)


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
model=BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: bert-base-uncased
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
classifier.weight                          | MISSING    | 
classifier.bias                            | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


In [ ]:
# for layer in model.bert.encoder.layer:
#   print(layer)

In [ ]:
training_args=TrainingArguments(
    output_dir='./bert-finetuned-imdb',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    logging_dir='./logs',
    learning_rate=2e-5,
    weight_decay=0.01,
    do_train=True,
    report_to='none'

)

`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


In [ ]:
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [ ]:
trainer.train()

Step,Training Loss
500,0.266289
1000,0.124984


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=1125, training_loss=0.18088711081610787, metrics={'train_runtime': 475.5749, 'train_samples_per_second': 18.924, 'train_steps_per_second': 2.366, 'total_flos': 1183999749120000.0, 'train_loss': 0.18088711081610787, 'epoch': 3.0})

In [ ]:
trainer.save_model('./bert-finetuned-imdb')
tokenizer.save_pretrained('./bert-finetuned-imdb')

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

('./bert-finetuned-imdb/tokenizer_config.json',
 './bert-finetuned-imdb/tokenizer.json')

In [ ]:
metrics=trainer.evaluate()

In [ ]:
print(metrics)

{'eval_loss': 0.5512442588806152, 'eval_runtime': 7.2824, 'eval_samples_per_second': 68.659, 'eval_steps_per_second': 8.651, 'epoch': 3.0}


# Prediction

In [ ]:
tokenizer=BertTokenizer.from_pretrained('/content/bert-finetuned-imdb')
model=BertForSequenceClassification.from_pretrained('/content/bert-finetuned-imdb')

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

In [ ]:
from transformers import pipeline
classifier=pipeline('text-classification',model=model,tokenizer=tokenizer)

In [26]:
data=[
    "I absolutely loved this movie! The performances were outstanding, the story was engaging, and the cinematography was breathtaking. It kept me hooked from start to finish, and I left the theater feeling inspired and uplifted. Definitely one of the best films I've seen this year.",
    "This film was a complete disappointment. The plot was predictable, the acting felt forced, and the pacing was painfully slow. I struggled to stay interested, and by the end I just wanted it to be over. Honestly, I wouldn’t recommend it to anyone.",
    'The movie was good',
    'The movie was bad'
]
for text in data:
    print(classifier(text))

[{'label': 'LABEL_1', 'score': 0.9992244243621826}]
[{'label': 'LABEL_0', 'score': 0.9991507530212402}]
[{'label': 'LABEL_1', 'score': 0.9955765008926392}]
[{'label': 'LABEL_0', 'score': 0.9985633492469788}]


In [27]:
print(result)

NameError: name 'result' is not defined

In [28]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("/content/bert-finetuned-imdb")
print(model.config.id2label)
print(model.config.label2id)

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

{0: 'LABEL_0', 1: 'LABEL_1'}
{'LABEL_0': 0, 'LABEL_1': 1}


In [41]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from huggingface_hub import whoami

print(whoami())
